In [15]:
!pip install transformers keybert

In [29]:
#java 오류시
#import os
#os.environ['JAVA_HOME'] = r'C:\Program Files\Java\jdk-23'  # 설치된 JDK 경로로 변경
#os.environ['PATH'] = os.environ['JAVA_HOME'] + r'\bin;' + os.environ['PATH']

In [17]:
url = 'https://n.news.naver.com/mnews/article/014/0005261392'

In [19]:
import json
import requests
from bs4 import BeautifulSoup
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from keybert import KeyBERT
import torch

# FinBERT 모델과 토크나이저 초기화
tokenizer = AutoTokenizer.from_pretrained("yiyanghkust/finbert-tone")
model = AutoModelForSequenceClassification.from_pretrained("yiyanghkust/finbert-tone")

# KeyBERT 초기화
kw_model = KeyBERT()

# 주어진 URL에서 기사 본문을 크롤링하는 함수
def crawl_article_text(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    article_content = soup.select_one('#dic_area')
    return article_content.get_text(separator=" ", strip=True) if article_content else None

# FinBERT를 사용하여 금융 관련 키워드 추출
def extract_financial_terms(text, top_n=10):
    # KeyBERT를 사용해 주요 키워드 추출
    keywords = kw_model.extract_keywords(text, keyphrase_ngram_range=(1, 2), stop_words='english', top_n=top_n)
    
    # FinBERT로 키워드를 검토하여 금융 용어 필터링
    financial_terms = []
    for kw in keywords:
        inputs = tokenizer(kw[0], return_tensors="pt")
        outputs = model(**inputs)
        scores = torch.nn.functional.softmax(outputs.logits, dim=-1)
        label = torch.argmax(scores).item()
        
        # 긍정(Positive), 부정(Negative), 중립(Neutral) 중 금융 관련성이 높은 용어를 추가
        if label == 0:  # 중립 (금융 용어 가능성이 높음)
            financial_terms.append(kw[0])

    return list(set(financial_terms))

# 노드와 관계를 구조화하여 JSON 형식으로 저장
def create_nodes_and_edges(terms):
    node_id = 1
    nodes = []
    edges = []

    term_map = {}
    for term in terms:
        # 노드 생성
        nodes.append({"id": node_id, "word": term})
        term_map[term] = node_id
        node_id += 1

    # 용어 간 임의 관계 추가 (예시)
    for i in range(len(terms)):
        for j in range(i + 1, len(terms)):
            edges.append({
                "source": term_map[terms[i]],
                "target": term_map[terms[j]],
                "relation_description": f"{terms[i]}와 {terms[j]}는 금융 기사에서 관련된 용어로 함께 등장합니다."
            })

    return {"nodes": nodes, "edges": edges}


# 기사 본문 크롤링
article_text = crawl_article_text(url)

# 본문이 없을 경우 종료
if not article_text:
    print(f"Article content not found for {url}")
else:
    # FinBERT 및 KeyBERT를 통해 금융 용어 추출
    financial_terms = extract_financial_terms(article_text)

    # 노드 및 관계 생성
    extracted_data = create_nodes_and_edges(financial_terms)

    # JSON 형식으로 저장
    output_path = 'financial_terms_extracted.json'
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(extracted_data, f, ensure_ascii=False, indent=4)

    print(f"Extracted data saved to {output_path}")


config.json:   0%|          | 0.00/533 [00:00<?, ?B/s]

C:\Users\sashi\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\sashi\.cache\huggingface\hub\models--yiyanghkust--finbert-tone. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/439M [00:00<?, ?B/s]

Extracted data saved to financial_terms_extracted.json
